# Projeto Final - Limpeza de Dados

## Setup

In [1]:
from pyspark.sql import functions as sf
import pyspark.sql.functions as f

# Modeling
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
## Bibliotecas Gráficas
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Criar a sessao do Spark
from pyspark.sql import SparkSession
spark = SparkSession \
            .builder \
            .master("local[4]") \
            .appName("nyc_caio_garcia") \
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.4,com.microsoft.azure:azure-storage:8.6.6") \
            .getOrCreate()

In [4]:
# Acesso aos dados na nuvem
STORAGE_ACCOUNT = 'dlspadseastusprod'
CONTAINER = 'big-data-comp-nuvem'
FOLDER = 'airline-delay'
TOKEN = 'lSuH4ZI9BhOFEhCF/7ZQbrpPBIhgtLcPDfXjJ8lMxQZjaADW4p6tcmiZGDX9u05o7FqSE2t9d2RD+ASt0YFG8g=='

spark.conf.set("fs.azure.account.key." + STORAGE_ACCOUNT + ".blob.core.windows.net", TOKEN)

### Schema
Schema definido de acordo com o dicionário de dados em `projeto_final_dicionário.xlsx`

In [5]:
from pyspark.sql.types import *

labels = (('FL_DATE', TimestampType()),
          ('OP_CARRIER', StringType()),
          ('OP_CARRIER_FL_NUM', IntegerType()),
          ('ORIGIN', StringType()),
          ('DEST', StringType()),
          ('CRS_DEP_TIME', IntegerType()),
          ('DEP_TIME', FloatType()),
          ('DEP_DELAY', FloatType()),
          ('TAXI_OUT', FloatType()),
          ('WHEELS_OFF', FloatType()),
          ('WHEELS_ON', FloatType()),
          ('TAXI_IN', FloatType()),
          ('CRS_ARR_TIME', IntegerType()),
          ('ARR_TIME', FloatType()),
          ('ARR_DELAY', FloatType()),
          ('CANCELLED', FloatType()),
          ('CANCELLATION_CODE', StringType()),
          ('DIVERTED', FloatType()),
          ('CRS_ELAPSED_TIME', FloatType()),
          ('ACTUAL_ELAPSED_TIME', FloatType()),
          ('AIR_TIME', FloatType()),
          ('DISTANCE', FloatType()),
          ('CARRIER_DELAY', FloatType()),
          ('WEATHER_DELAY', FloatType()),
          ('NAS_DELAY', FloatType()),
          ('SECURITY_DELAY', FloatType()),
          ('LATE_AIRCRAFT_DELAY', StringType()))

schema = StructType([StructField(x[0], x[1], True) for x in labels])

In [6]:
# Columns with values in minutes
minute_columns = ["TAXI_OUT","TAXI_IN","DEP_DELAY","ARR_DELAY","AIR_TIME","CRS_ELAPSED_TIME","ACTUAL_ELAPSED_TIME",
                  "CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]

# Subset from minute columns with no data leak from moment of take off
clean_min_columns = ["TAXI_OUT", "DEP_DELAY", "CRS_ELAPSED_TIME"]

# Columns with time information on format 'hhmm'
# Not proper for numerical manipulation
odd_format_columns = ["CRS_DEP_TIME","DEP_TIME","WHEELS_OFF","WHEELS_ON","ARR_TIME","CRS_ARR_TIME"]

### Carregamento de dados
Dados carregados da nuvem como spark data frame

Exemplo para o ano de 2009:

In [7]:
config = spark.sparkContext._jsc.hadoopConfiguration()
config.set("fs.azure.account.key." + STORAGE_ACCOUNT + ".blob.core.windows.net", TOKEN)
sc = spark.sparkContext

df_exemple = spark.read.csv("wasbs://{}@{}.blob.core.windows.net/{}/2009.csv"\
                    .format(CONTAINER, STORAGE_ACCOUNT, FOLDER), header=True, schema=schema)
df_exemple.take(2)

[Row(FL_DATE=datetime.datetime(2009, 1, 1, 0, 0), OP_CARRIER='XE', OP_CARRIER_FL_NUM=1204, ORIGIN='DCA', DEST='EWR', CRS_DEP_TIME=1100, DEP_TIME=1058.0, DEP_DELAY=-2.0, TAXI_OUT=18.0, WHEELS_OFF=1116.0, WHEELS_ON=1158.0, TAXI_IN=8.0, CRS_ARR_TIME=1202, ARR_TIME=1206.0, ARR_DELAY=4.0, CANCELLED=0.0, CANCELLATION_CODE=None, DIVERTED=0.0, CRS_ELAPSED_TIME=62.0, ACTUAL_ELAPSED_TIME=68.0, AIR_TIME=42.0, DISTANCE=199.0, CARRIER_DELAY=None, WEATHER_DELAY=None, NAS_DELAY=None, SECURITY_DELAY=None, LATE_AIRCRAFT_DELAY=None),
 Row(FL_DATE=datetime.datetime(2009, 1, 1, 0, 0), OP_CARRIER='XE', OP_CARRIER_FL_NUM=1206, ORIGIN='EWR', DEST='IAD', CRS_DEP_TIME=1510, DEP_TIME=1509.0, DEP_DELAY=-1.0, TAXI_OUT=28.0, WHEELS_OFF=1537.0, WHEELS_ON=1620.0, TAXI_IN=4.0, CRS_ARR_TIME=1632, ARR_TIME=1624.0, ARR_DELAY=-8.0, CANCELLED=0.0, CANCELLATION_CODE=None, DIVERTED=0.0, CRS_ELAPSED_TIME=82.0, ACTUAL_ELAPSED_TIME=75.0, AIR_TIME=43.0, DISTANCE=213.0, CARRIER_DELAY=None, WEATHER_DELAY=None, NAS_DELAY=None, SEC

Como temos dados para os anos de 2009 até o ano de 2018. Iremos criar um dicionario contendo os dataframes de cada ano separadamente. 

In [8]:
# Criando dicionario de dataframes
df_for_year = {}

# Loop lendo arquivo de cada ano e salvando no dicionario
for year in range(2009, 2019):
    # Ajustando o caminho
    file_path = "wasbs://{}@{}.blob.core.windows.net/{}/{}.csv"\
                    .format(CONTAINER, STORAGE_ACCOUNT, FOLDER, year)
    
    # lendo arquivo csv 
    df_name = "df_{}".format(year)
    df = spark.read.csv(file_path, header=True, schema=schema)
    
    # Adicionando df ao dicionario de dataframes 
    df_for_year[df_name] = df
    

# Visualizando as primeiras linhas de 2012
df_for_year["df_2012"].take(5)

[Row(FL_DATE=datetime.datetime(2012, 1, 1, 0, 0), OP_CARRIER='MQ', OP_CARRIER_FL_NUM=4041, ORIGIN='BMI', DEST='ORD', CRS_DEP_TIME=None, DEP_TIME=1702.0, DEP_DELAY=-13.0, TAXI_OUT=9.0, WHEELS_OFF=1711.0, WHEELS_ON=1748.0, TAXI_IN=9.0, CRS_ARR_TIME=None, ARR_TIME=1757.0, ARR_DELAY=-13.0, CANCELLED=0.0, CANCELLATION_CODE=None, DIVERTED=0.0, CRS_ELAPSED_TIME=55.0, ACTUAL_ELAPSED_TIME=55.0, AIR_TIME=37.0, DISTANCE=116.0, CARRIER_DELAY=None, WEATHER_DELAY=None, NAS_DELAY=None, SECURITY_DELAY=None, LATE_AIRCRAFT_DELAY=None),
 Row(FL_DATE=datetime.datetime(2012, 1, 1, 0, 0), OP_CARRIER='MQ', OP_CARRIER_FL_NUM=4041, ORIGIN='ORD', DEST='BMI', CRS_DEP_TIME=None, DEP_TIME=1540.0, DEP_DELAY=-5.0, TAXI_OUT=11.0, WHEELS_OFF=1551.0, WHEELS_ON=1614.0, TAXI_IN=5.0, CRS_ARR_TIME=None, ARR_TIME=1619.0, ARR_DELAY=-16.0, CANCELLED=0.0, CANCELLATION_CODE=None, DIVERTED=0.0, CRS_ELAPSED_TIME=50.0, ACTUAL_ELAPSED_TIME=39.0, AIR_TIME=23.0, DISTANCE=116.0, CARRIER_DELAY=None, WEATHER_DELAY=None, NAS_DELAY=None, 

Com todos os dataframes pré-importados(lazy) podemos realizar um merge unindo todos os anos.

In [9]:
# Importando função reduce para realizar o merge
from functools import reduce 

In [10]:
# União de todos os DataFrames em um único Data frame
df_final = reduce(sf.DataFrame.union, df_for_year.values())

# Criando a coluna "year" baseada na coluna "date"
df_final = df_final.withColumn("year", sf.year("FL_DATE"))

# Exibindo as primeiras linhas
df_final.take(10)

[Row(FL_DATE=datetime.datetime(2009, 1, 1, 0, 0), OP_CARRIER='XE', OP_CARRIER_FL_NUM=1204, ORIGIN='DCA', DEST='EWR', CRS_DEP_TIME=1100, DEP_TIME=1058.0, DEP_DELAY=-2.0, TAXI_OUT=18.0, WHEELS_OFF=1116.0, WHEELS_ON=1158.0, TAXI_IN=8.0, CRS_ARR_TIME=1202, ARR_TIME=1206.0, ARR_DELAY=4.0, CANCELLED=0.0, CANCELLATION_CODE=None, DIVERTED=0.0, CRS_ELAPSED_TIME=62.0, ACTUAL_ELAPSED_TIME=68.0, AIR_TIME=42.0, DISTANCE=199.0, CARRIER_DELAY=None, WEATHER_DELAY=None, NAS_DELAY=None, SECURITY_DELAY=None, LATE_AIRCRAFT_DELAY=None, year=2009),
 Row(FL_DATE=datetime.datetime(2009, 1, 1, 0, 0), OP_CARRIER='XE', OP_CARRIER_FL_NUM=1206, ORIGIN='EWR', DEST='IAD', CRS_DEP_TIME=1510, DEP_TIME=1509.0, DEP_DELAY=-1.0, TAXI_OUT=28.0, WHEELS_OFF=1537.0, WHEELS_ON=1620.0, TAXI_IN=4.0, CRS_ARR_TIME=1632, ARR_TIME=1624.0, ARR_DELAY=-8.0, CANCELLED=0.0, CANCELLATION_CODE=None, DIVERTED=0.0, CRS_ELAPSED_TIME=82.0, ACTUAL_ELAPSED_TIME=75.0, AIR_TIME=43.0, DISTANCE=213.0, CARRIER_DELAY=None, WEATHER_DELAY=None, NAS_DELA

Assim, temos o dataframe com todos os anos.

In [11]:
OBSERVACOES = df_final.count()
assert (OBSERVACOES == 61556964)

In [12]:
OBSERVACOES

61556964

A base contem 61556964 observações.

In [13]:
CANCELAMENTOS = df_final.filter(df_final.CANCELLED == 1).count()
assert (CANCELAMENTOS == 973209)

In [14]:
CANCELAMENTOS

973209

Dos voos na base, 973209 foram cancelados.

## Tratamento de dados faltantes

Das 27 colunas na base de dados, 16 tem valores faltantes. Os dados ausentes na coluna CANCELATION_CODE sao 100% consistentes com a informacao de cancelamento, isto eh, apenas os voos que nao foram cancelados tem a coluna CANCELATION_CODE vazia. As outras 15 colunas podem ser agrupadas em 3 grupos: _Voo_, _Chegada_ e _Atrasos_.

### Resumo

In [50]:
missing_counts = df_final.select([sf.col(column).isNull().cast("int").alias(column) for column in df_final.columns]) \
                       .groupBy() \
                       .sum()

In [51]:
# Criando dataframe de colunas com valores zerados
missing_counts_df = missing_counts.toPandas().transpose()

# Filtrando apenas colunas com valores nulos
missing_counts_df = missing_counts_df[missing_counts_df[0]>0]

# Renomeando a coluna
missing_counts_df = missing_counts_df.rename(columns={0:"nulos"})

In [52]:
# Contando número de colunas com valores nulos
print("Número de colunas com valores faltantes:")
missing_counts_df.count()[0]

Número de colunas com valores faltantes:


19

In [53]:
# Cálculando porcentagem de valores faltantes
missing_counts_df["%nulos"] = (missing_counts_df["nulos"]/OBSERVACOES) * 100

# Ordenando por % de nulos
missing_counts_df = missing_counts_df.sort_values("%nulos", ascending=False)

# Visualizando resultados
missing_counts_df

,nulos,%nulos
sum(CANCELLATION_CODE),60583755,98.419011
sum(LATE_AIRCRAFT_DELAY),50166224,81.495611
sum(SECURITY_DELAY),50166224,81.495611
sum(NAS_DELAY),50166224,81.495611
sum(WEATHER_DELAY),50166224,81.495611
sum(CARRIER_DELAY),50166224,81.495611
sum(CRS_ARR_TIME),6096762,9.904260
sum(CRS_DEP_TIME),6096762,9.904260
sum(ARR_DELAY),1121351,1.821648
sum(ACTUAL_ELAPSED_TIME),1118754,1.817429


In [ ]:
# Visualizando missing em gráfico de barras
# Ajustando o tamanho da figura
plt.figure(figsize=(10, 4))

# Plotando o gráfico de barras
sns.barplot(x=missing_counts_df.index, y=missing_counts_df["%nulos"], color="red")

# Adicionando inclinação aos valores do eixo x
plt.xticks(rotation=45, ha='right')

# Adicionando título e rótulos aos eixos
plt.title('Gráfico de Barras')
plt.xlabel("Colunas")
plt.ylabel("%Nulos")

# Exibindo o gráfico
plt.show()

Conforme evidenciado no gráfico apresentado, nota-se que a coluna "CANCELLATION_CODE" exibe uma lacuna em praticamente 100% dos dados, enquanto as colunas "LATE_AIRCRAFT_DELAY", "NAS_DELAY", "WEATHER_DELAY", "CARRIER_DELAY" e "SECURITY_DELAY" apresentam uma ausência de informações em torno de 80%.

### Cancelamentos

In [ ]:
assert (df_final.filter((df_final.CANCELLATION_CODE.isNull()) &
                  (df_final.CANCELLED == 0)).count() ==
        df_final.filter(df_final.CANCELLED == 0).count())

Todos os valores faltantes de CANCELLATION_CODE são referentes a voos que não foram cancelados.

### Voo

#### Testes

`DEP_TIME` e `DEP_DELAY`: co-ausentes, todos cancelados

In [ ]:
assert (df_final.filter((df_final.DEP_TIME.isNull())   &
                  (df_final.DEP_DELAY.isNull())  &
                  (df_final.TAXI_OUT.isNull())   &
                  (df_final.WHEELS_OFF.isNull()) &
                  (df_final.WHEELS_ON.isNull())  &
                  (df_final.TAXI_IN.isNull())    &
                  (df_final.ARR_TIME.isNull())).count() ==
        df_final.filter(df_final.DEP_TIME.isNull()).count())

assert (df_final.filter((df_final.DEP_TIME.isNull())   &
                  (df_final.DEP_DELAY.isNull())).count() ==
        df_final.filter(df_final.DEP_TIME.isNull()).count())

In [ ]:
assert (df_final.filter((df_final.DEP_TIME.isNull())   &
                  (df_final.CANCELLED == 1)).count() ==
        df_final.filter(df_final.DEP_TIME.isNull()).count())

`TAXI_OUT` e `WHEELS_OFF`: co-ausentes e cancelados

In [ ]:
assert (df_final.filter((df_final.TAXI_OUT.isNull())   &
                  (df_final.WHEELS_OFF.isNull()) &
                  (df_final.WHEELS_ON.isNull())  &
                  (df_final.TAXI_IN.isNull())    &
                  (df_final.ARR_TIME.isNull())).count() ==
        df_final.filter(df_final.TAXI_OUT.isNull()).count())

assert (df_final.filter((df_final.TAXI_OUT.isNull())   &
                  (df_final.WHEELS_OFF.isNull())).count() ==
        df_final.filter(df_final.TAXI_OUT.isNull()).count())

In [ ]:
assert (df_final.filter((df_final.TAXI_OUT.isNull())   &
                  (df_final.CANCELLED == 1)).count() ==
       df_final.filter(df_final.TAXI_OUT.isNull()).count())

`WHEELS_ON`, `TAXI_IN` e `ARR_TIME`: co-ausentes

In [ ]:
assert (df_final.filter((df_final.WHEELS_ON.isNull())  &
                  (df_final.TAXI_IN.isNull())    &
                  (df_final.ARR_TIME.isNull())).count() ==
        df_final.filter(df_final.TAXI_IN.isNull()).count())

In [ ]:
assert (df_final.filter((df_final.TAXI_IN.isNull())   &
                  (df_final.CANCELLED == 1)).count() ==
        df_final.filter(df_final.CANCELLED == 1).count())

In [ ]:
assert (df.filter((df.TAXI_IN.isNull()) &
                  (df.CANCELLED == 0)   &
                  (df.DIVERTED == 1)).count() ==
        df.filter((df.TAXI_IN.isNull()) &
                  (df.CANCELLED == 0)).count())

#### Análise

O grupo _Voo_ apresenta uma relação entre voos cancelados e as 7 variáveis:
 - DEP_TIME
 - DEP_DELAY
 - TAXI_OUT
 - WHEELS_OFF
 - WHEELS_ON
 - TAXI_IN
 - ARR_TIME
 
Os valores faltantes para WHEELS_ON, TAXI_IN e ARR_TIME coincidem nas mesmas observações (com uma exceção descrita mais abaixo). Todos os voos cancelados se encontram dentre essas observações. Os valores faltantes para TAXI_OUT e WHEELS_OFF coincidem nas mesmas observações, todas referentes a voos cancelados. Finalmente, os valores faltantes de DEP_TIME e DEP_DELAY coincidem nas mesmas observações, todas com valores faltantes para TAXI_OUT.

Todos os voos que não foram cancelados mas não tem informação da hora de aterrisagem (`(df.WHEELS_ON.isNull()) & (df.CANCELLED == 0)`) foram redirecionados para um aeroporto diferente do aeroporto destino original (`df.DIVERTED == 1`)

Destas relações, supomos:
 - A diferença entre DEP_TIME e WHEELS_OFF pode ser devido a voos que chegam a sair do chão antes de serem cancelados, e voos que são cancelados após o embarque mas antes da decolagem.
 - Nenhum desses valores faltantes parece implausível o suficiente para assumirmos erro nos dados baseado apenas nessa análise. Alguns desses dados podem vir a ser retirados mesmo assim por questão de propriedades dos algorítmos utilizados mais a frente.

### Chegada

### Atrasos
TODO

#### Testes

In [ ]:
assert (df.filter((df.CANCELLED == 0) &
                  (df.DEP_DELAY > 0)).count() == 
        2252608)

In [ ]:
assert (df.filter((df.CANCELLED == 0) &
                  (df.ARR_DELAY > 0)).count() ==
        2402990)

In [ ]:
assert (df.filter((df.CANCELLED == 0) &
                  ((df.DEP_DELAY > 0) |
                   (df.ARR_DELAY > 0))).count() ==
        3052688)

In [ ]:
assert (OBSERVACOES - df.filter(df.CARRIER_DELAY.isNull()).count() ==
        1170501)

In [ ]:
assert (df.filter((df.CARRIER_DELAY.isNull()) &
                  (df.WEATHER_DELAY.isNull()) &
                  (df.NAS_DELAY.isNull()) &
                  (df.SECURITY_DELAY.isNull()) &
                  (df.LATE_AIRCRAFT_DELAY.isNull())).count() ==
        df.filter(df.CARRIER_DELAY.isNull()).count())

In [ ]:
df.filter(df.CARRIER_DELAY == 0).select(df.OP_CARRIER_FL_NUM, df.CARRIER_DELAY).take(10)

#### Análise

Todos os dados faltantes referentes a categoria de atraso coincidem nas mesmas observações.

Há menos observações com informação sobre a causa do atraso do que voos atrasados, independente se medindo o atraso de saída ou de chegada.

### Anomalia

In [ ]:
assert (df.filter((df.ACTUAL_ELAPSED_TIME.isNull()) &
                  (df.AIR_TIME == 0)                &
                  (df.WHEELS_ON.isNull())           &
                  (df.ARR_TIME.isNull())            &
                  (df.ARR_DELAY == 0)               &
                  (df.TAXI_IN == 0)                 &
                  (df.CANCELLED == 0)).count() == 1)

Uma mesma observação é responsavel pela discrepância na quantidade total de valores faltantes entre WHEELS_ON, TAXI_IN e ARR_TIME, e ARR_DELAY, ACTUAL_ELAPSED_TIME e AIR_TIME.
Um valor de `AIR_TIME == 0` nao faz sentido para um voo que não foi cancelado, e o mesmo se aplica a `TAXI_IN == 0`. Ao retirar essa observação da base, a análise de dados faltantes por grupo torna-se mais consistente.

## Consistencia

In [ ]:
assert (df.filter(df.AIR_TIME + df.TAXI_IN + df.TAXI_OUT != df.ACTUAL_ELAPSED_TIME).count() == 0)

In [ ]:
assert (df.filter((df.CANCELLED == 1) &
                  (df.DIVERTED == 1)).count() == 0)

In [ ]:
assert (df.filter((df.DEP_TIME % 1 != 0) | (df.DEP_DELAY % 1 != 0)).count() == 0)

# Modelagem

In [12]:
# This list includes all values not known at the moment of takeoff
# except `ARR_DELAY` which will be used as target variable
take_off_leak = ["WHEELS_ON","TAXI_IN","ARR_TIME","ACTUAL_ELAPSED_TIME","AIR_TIME",
                 "CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]

In [13]:
take_off_df = df_final.drop(*take_off_leak)\
                      .filter(df_final.CANCELLED == 0)\
                      .filter(df_final.DIVERTED == 0)\
                      .filter(df_final.CRS_ARR_TIME.isNotNull())\
                      .filter(df_final.ARR_DELAY.isNotNull())\
                      .filter(df_final.DEP_DELAY.isNotNull())

In [14]:
missing_counts = take_off_df.select([sf.col(column).isNull().cast("int").alias(column) for column in take_off_df.columns]) \
                       .groupBy() \
                       .sum()

In [15]:
# Criando dataframe de colunas com valores zerados
missing_counts_df = missing_counts.toPandas().transpose()

# Filtrando apenas colunas com valores nulos
missing_counts_df = missing_counts_df[missing_counts_df[0]>0]

# Renomeando a coluna
missing_counts_df = missing_counts_df.rename(columns={0:"nulos"})

In [16]:
# Cálculando porcentagem de valores faltantes
missing_counts_df["%nulos"] = (missing_counts_df["nulos"]/OBSERVACOES) * 100

# Ordenando por % de nulos
missing_counts_df = missing_counts_df.sort_values("%nulos", ascending=False)

# Visualizando resultados
missing_counts_df

,nulos,%nulos
sum(CANCELLATION_CODE),54425644,88.415088
sum(CRS_ELAPSED_TIME),1,0.000002


## Train/Test Split

In [17]:
train_df, test_df = take_off_df.randomSplit([0.8,0.2], seed=42)
toy_df = train_df.sample(False, 0.01, seed=42)

In [ ]:
print("Train set count:", train_df.count())
print("Test set count:", test_df.count())
print("Toy set count:", toy_df.count())

## Feature Engineering: One-Hot-Enconding

In [18]:
cat_features = ["OP_CARRIER", "OP_CARRIER_FL_NUM", "ORIGIN", "DEST"]

indexOutputCols = [x + 'Index' for x in cat_features]

oheOutputCols = [x + 'OHE' for x in cat_features]

stringIndex = StringIndexer(inputCols = cat_features,
                            outputCols = indexOutputCols,
                            handleInvalid = 'skip')

oheEncoder = OneHotEncoder(inputCols = indexOutputCols,
                           outputCols = oheOutputCols)

In [19]:
num_features = ["TAXI_OUT", "DEP_DELAY", "CRS_ELAPSED_TIME", "DISTANCE"]

numVecAssembler = VectorAssembler(inputCols = num_features,
                                  outputCol = 'features',
                                  handleInvalid = 'skip')

stdScaler = StandardScaler(inputCol = 'features',
                           outputCol = 'features_scaled')

## Assembling dos vetores

In [20]:
assembleInputs = oheOutputCols + ['features_scaled']

vecAssembler = VectorAssembler(inputCols = assembleInputs,
                               outputCol = 'features_vector')

In [21]:
stages = [stringIndex, oheEncoder, numVecAssembler, stdScaler, vecAssembler]

## Criação do Pipeline

In [22]:
# Criacao do pipeline de transformacao
transform_pipeline = Pipeline(stages=stages)

# Aplicacao do pipeline nos dados de treino
fitted_transformer = transform_pipeline.fit(train_df)
transformed_train_df = fitted_transformer.transform(train_df)

transformed_train_df.limit(10).toPandas()

/opt/conda/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,OP_CARRIER_FL_NUMIndex,ORIGINIndex,DESTIndex,OP_CARRIEROHE,OP_CARRIER_FL_NUMOHE,ORIGINOHE,DESTOHE,features,features_scaled,features_vector
0,2009-01-01,9E,2102,AUS,IND,1655,1650.0,-5.0,19.0,1709.0,...,2722.0,36.0,48.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[19.0, -5.0, 155.0, 920.0]","[2.0237826918319213, -0.13391154613205383, 2.1...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2009-01-01,9E,2103,IND,AUS,1445,1450.0,5.0,14.0,1504.0,...,2374.0,48.0,36.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[14.0, 5.0, 150.0, 920.0]","[1.4912082992445737, 0.13391154613205383, 2.03...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2009-01-01,9E,2109,MSP,OKC,2130,2126.0,-4.0,34.0,2200.0,...,2747.0,11.0,61.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[34.0, -4.0, 133.0, 695.0]","[3.621505869593965, -0.10712923690564306, 1.80...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,2009-01-01,9E,2110,OKC,MSP,1745,1740.0,-5.0,8.0,1748.0,...,2119.0,61.0,11.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[8.0, -5.0, 132.0, 695.0]","[0.8521190281397564, -0.13391154613205383, 1.7...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,2009-01-01,9E,2114,ALO,MSP,700,815.0,75.0,16.0,831.0,...,2233.0,306.0,11.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[16.0, 75.0, 69.0, 166.0]","[1.7042380562795127, 2.0086731919808076, 0.938...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,2009-01-01,9E,2118,MSP,GRR,1310,1337.0,27.0,14.0,1351.0,...,1901.0,11.0,77.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[14.0, 27.0, 93.0, 408.0]","[1.4912082992445737, 0.7231223491130907, 1.264...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,2009-01-01,9E,2122,CLE,MSP,1343,1338.0,-5.0,11.0,1349.0,...,2105.0,38.0,11.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[11.0, -5.0, 135.0, 622.0]","[1.171663663692165, -0.13391154613205383, 1.83...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,2009-01-01,9E,2122,MSP,RHI,1538,1549.0,11.0,12.0,1601.0,...,2105.0,11.0,273.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[12.0, 11.0, 58.0, 190.0]","[1.2781785422096346, 0.29460540149051845, 0.78...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,2009-01-01,9E,2123,DTW,GRR,1218,1214.0,-4.0,16.0,1230.0,...,2532.0,9.0,77.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[16.0, -4.0, 62.0, 120.0]","[1.7042380562795127, -0.10712923690564306, 0.8...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

## Model Training

In [23]:
model = LinearRegression(maxIter = 5, # pode causar overfitting
                         solver = 'auto',
                         labelCol = 'ARR_DELAY',
                         featuresCol = 'features_vector',
                         elasticNetParam = 0.2,
                         regParam = 0.02)

pipe_stages = stages + [model]

pipe = Pipeline(stages=pipe_stages)

In [24]:
fitted_pipe = pipe.fit(toy_df)

## Model performance evaluation

In [25]:
preds = fitted_pipe.transform(test_df)

In [26]:
preds.limit(10).toPandas()

/opt/conda/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,ORIGINIndex,DESTIndex,OP_CARRIEROHE,OP_CARRIER_FL_NUMOHE,ORIGINOHE,DESTOHE,features,features_scaled,features_vector,prediction
0,2009-01-01,9E,2108,OKC,MSP,700,650.0,-10.0,15.0,705.0,...,61.0,11.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[15.0, -10.0, 135.0, 695.0]","[1.592625007550886, -0.2683625025834257, 1.834...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-15.030487
1,2009-01-01,9E,2115,MSP,ALO,2245,2245.0,0.0,31.0,2316.0,...,11.0,316.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[31.0, 0.0, 58.0, 166.0]","[3.2914250156051645, 0.0, 0.7881109483871731, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",9.436105
2,2009-01-01,9E,2120,STL,MSP,1610,1552.0,-18.0,7.0,1559.0,...,29.0,11.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7.0, -18.0, 110.0, 449.0]","[0.7432250035237469, -0.4830525046501663, 1.49...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-29.290478
3,2009-01-01,9E,2125,MSP,CLE,1015,1014.0,-1.0,23.0,1037.0,...,11.0,40.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[23.0, -1.0, 117.0, 622.0]","[2.4420250115780253, -0.026836250258342572, 1....","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-5.115942
4,2009-01-01,9E,2138,SAT,IND,1915,1916.0,1.0,14.0,1930.0,...,43.0,47.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[14.0, 1.0, 164.0, 986.0]","[1.4864500070474937, 0.026836250258342572, 2.2...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-6.717591
5,2009-01-01,9E,2143,CMX,MSP,545,550.0,5.0,17.0,607.0,...,262.0,11.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[17.0, 5.0, 82.0, 277.0]","[1.8049750085576708, 0.13418125129171285, 1.11...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.606369
6,2009-01-01,9E,2149,MSP,GRB,1259,1259.0,0.0,11.0,1310.0,...,11.0,119.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[11.0, 0.0, 70.0, 252.0]","[1.1679250055373165, 0.0, 0.9511683859845192, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-10.853533
7,2009-01-01,9E,2156,AZO,MSP,710,711.0,1.0,19.0,730.0,...,183.0,11.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[19.0, 1.0, 108.0, 426.0]","[2.0173250095644555, 0.026836250258342572, 1.4...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-2.881877
8,2009-01-01,9E,2168,BDL,IND,1620,1626.0,6.0,18.0,1644.0,...,56.0,47.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[18.0, 6.0, 155.0, 728.0]","[1.9111500090610631, 0.16101750155005543, 2.10...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [27]:
rmse = RegressionEvaluator(labelCol = 'ARR_DELAY',
                           metricName = 'rmse').evaluate(preds)

In [28]:
print("RMSEof Prediction on test set:", rmse)

RMSEof Prediction on test set: 10.060889556975196


In [32]:
results = []
results.append({"run": 1,
                "rmse": rmse,
                "model": "LinearRegression",
                "params": "maxIter = 5, solver = 'auto', labelCol = 'ARR_DELAY', featuresCol = 'features_vector', elasticNetParam = 0.2, regParam = 0.02"})

In [29]:
df_final.drop(*take_off_leak)\
                      .filter(df_final.CANCELLED == 0)\
                      .filter(df_final.DIVERTED == 0)\
                      .filter(df_final.CRS_ARR_TIME.isNotNull())\
                      .filter(df_final.ARR_DELAY.isNotNull())\
                      .filter(df_final.DEP_DELAY.isNotNull()).count()

54425644